In [1]:
import pandas as pd
import numpy as np
import csv

def get_data():
    data = pd.read_csv('datasets/characteristics.csv', encoding='big5') # 从csv文件读全部数据
    x_data = data.iloc[:, 73:91].to_numpy()
    y_data = data.iloc[:, -1].to_numpy()[:, np.newaxis]
    
    train_num = int(x_data.shape[0] * 0.9)
    x_train = x_data[:train_num, :]
    x_val = x_data[train_num:, :]
    y_train = y_data[:train_num, :]
    y_val = y_data[train_num:, :]
    
    print('x_train shape: ', x_train.shape)
    print('y_train shape: ', y_train.shape)
    return x_train, y_train, x_val, y_val
x_train, y_train, x_val, y_val = get_data()

x_train shape:  (5086, 18)
y_train shape:  (5086, 1)


In [2]:
from data_utils import *

data_dir = 'datasets/'
raw_train_data, raw_test_data = get_data(data_dir)     # 1. 从数据集内读数据

def label_preprocessing(raw_test_data):
    
    x_test = np.empty([240, 18*9], dtype=float)
    for i in range(240):
        x_test[i, :] = raw_test_data[18*i:18*(i+1), :].reshape(1, -1)
    x_test = x_test[:, 72:90]
    print('x_test shape: ', x_test.shape)
    return x_test
x_test = label_preprocessing(raw_test_data)

raw_train_data shape:  (4320, 24)
raw_test_data shape:  (4320, 9)
x_test shape:  (240, 18)


In [3]:
# 前向传播的构建
import tensorflow as tf

input_size = x_train.shape[1]
layer1_size, layer2_size = 24, 12
output_size = 1

def get_weight_variable(shape, regularizer):
    weights = tf.get_variable('weights', shape, initializer=tf.truncated_normal_initializer(stddev=0.002))
    if regularizer != None:
        tf.add_to_collection('loss', regularizer(weights))
    return weights

def forward_propagation(x, regularizer):
    
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([input_size, layer1_size], regularizer)
        biases = tf.get_variable('biases', [layer1_size], initializer=tf.constant_initializer(0.0))
    layer1 = tf.nn.relu(tf.matmul(tf.cast(x, tf.float32), weights) + biases)
    
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([layer1_size, layer2_size], regularizer)
        biases = tf.get_variable('biases', [layer2_size], initializer=tf.constant_initializer(0.0))
    layer2 = tf.nn.relu(tf.matmul(tf.cast(layer1, tf.float32), weights) + biases)
    
    with tf.variable_scope('layer3'):
        weights = get_weight_variable([layer2_size, output_size], regularizer)
        biases = tf.get_variable('biases', [output_size], initializer=tf.constant_initializer(0.0))
    layer3 = tf.nn.relu(tf.matmul(tf.cast(layer2, tf.float32), weights) + biases)
    
    return layer3

D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [4]:
batch_size = 100
learning_rate_base = 2e-5
learning_rate_decay = 0.999
moving_average_decay = 0.99
reg_rate = 5e-7
iterations = 3e4

model_path = 'model.ckpt'

def train(x_train, y_train, x_val, y_val):
    
    x = tf.placeholder(tf.float32, [None, input_size], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 1], name='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(reg_rate)
    y_pred = forward_propagation(x, regularizer)
    
    print(y_pred.shape)
    
    global_step = tf.Variable(0, trainable=False)
    
    variable_average = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
    variable_average_op = variable_average.apply(tf.trainable_variables())
    
    loss = tf.reduce_mean(tf.square(y_ - y_pred))
    learning_rate = tf.train.exponential_decay(learning_rate_base, global_step,
                                    x_train.shape[0] / batch_size, learning_rate_decay)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variable_average_op]):
        train_op = tf.no_op(name='train')
        
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(int(iterations)):
            X_batch, y_batch = None, None
            idx = np.random.choice(x_train.shape[0], batch_size, replace=True)
            X_batch = x_train[idx]
            y_batch = y_train[idx]
            _, loss_value, step = sess.run([train_op, loss, global_step],
                                           feed_dict={x:X_batch, y_:y_batch})
            
            if i % 1000 == 0:
                print('After %d iterations, loss on x_train is: %g' %(step, loss_value))
                
            if i % 1000 == 0:
                val_dict = {x: x_val, y_: y_val}
                _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict=val_dict)
                print('After %d iterations, mse on x_val is: %g' % (i, loss_value))
        print('y_pred shape: ', y_pred.shape)
        result = sess.run(y_pred, feed_dict={x: x_test})
        print(result.shape)
        return result
#         print('y_pred: ',sess.run(y_pred,feed_dict={x: x_val}))

In [5]:
result = train(x_train, y_train, x_val, y_val)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

(?, 1)
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
After 1 iterations, loss on x_train is: 1025.06
After 0 iterations, mse on x_val is: 556.514
After 1002 iterations, loss on x_train is: 528.546
After 1000 iterations, mse on x_val is: 525.598
After 2003 iterations, loss on x_train is: 73.4921
After 2000 iterations, mse on x_val is: 62.0392
After 3004 iterations, loss on x_train is: 37.1382
After 3000 iterations, mse on x_val is: 42.87
After 4005 iterations, loss on x_train is: 27.1956
After 4000 iterations, mse on x_val is:

In [6]:
def result_output(y_pred):
    with open('outputs/submit_version4.csv', mode='w', newline='') as submit_file:
        csv_writer = csv.writer(submit_file)
        header = ['id', 'value']
        # print(header)
        csv_writer.writerow(header)
        for i in range(240):
            row = ['id_' + str(i), y_pred[i][0]]
            csv_writer.writerow(row)
            # print(row)
    print('submission.csv file saved.')
    
result_output(result)

submission.csv file saved.
